In [1]:
import pandas as pd
import datetime
from os import listdir
import os, os.path
from os.path import isfile, join
import numpy as np
import csv
import re
import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.util import dataframe_utils
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg

In [4]:
df_prueba = pd.read_csv("Log of analysis of time aspects.csv",index_col=0)
df_prueba

,case:id,case:concept:name,concept:name,Variant,time:timestamp,Question,Report,Year,Category,Type,Tupla,Organization
0,A1 P2 BPI 2020,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-11-30 12:54:15.489743,P2,A1,2020,Academic,Analyze Time,A12020,"Technische Universität, Berlin"
1,A1 P2 BPI 2020,A1 P2 BPI 2020,Compare cycle time,Compare cycle time,2021-11-30 13:54:15.489743,P2,A1,2020,Academic,Analyze Time,A12020,"Technische Universität, Berlin"
2,A2 P2 BPI 2020,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-11-30 14:54:15.489743,P2,A2,2020,Student,Analyze Time,A22020,"Faculty of Science, Utrecht University"
3,A2 P2 BPI 2020,A2 P2 BPI 2020,Compare cycle time,Compare cycle time,2021-11-30 15:54:15.489743,P2,A2,2020,Student,Analyze Time,A22020,"Faculty of Science, Utrecht University"
4,A7 P2 BPI 2020,A7 P2 BPI 2020,Calculate throughput,Calculate throughput,2021-11-30 16:54:15.489743,P2,A7,2020,Academic,Analyze Time,A72020,German Research Center for Artificial Intellig...
...,...,...,...,...,...,...,...,...,...,...,...,...
943,A19 P1 BPI 2020,A19 P1 BPI 2020,Group traces,Group traces by year,2022-01-08 19:54:15.489743,P1,A19,2020,Professional,Manipulation,A192020,Sberbank
944,A19 P1 BPI 2020,A19 P1 BPI 2020,Group traces,Group traces by activities,2022-01-08 20:54:15.489743,P1,A19,2020,Professional,Manipulation,A192020,Sberbank
945,A19 P1 BPI 2020,A19 P1 BPI 2020,Calculate statistics,Calculate Statistics of activities,2022-01-08 21:54:15.489743,P1,A19,2020,Professional,Calculate statistics,A192020,Sberbank
946,A19 P1 BPI 2020,A19 P1 BPI 2020,Calculate throughput,Calculate throughput,2022-01-08 22:54:15.489743,P1,A19,2020,Professional,Analyze Time,A192020,Sberbank


In [7]:
df_report_year = df_prueba.groupby(by=['Question', 'Year'])
for name, group in df_report_year:
    operations = group['concept:name'].to_list()
    

In [8]:
df_agrupado = pd.DataFrame(columns=['case:concept:name','Operations','Organization', 'Year', 'Question'])

In [9]:
df_analisis = df_prueba.groupby(by=['case:concept:name'])
# columns=['case:concept:name','Operations','Organization', 'Year', 'Question']
df_agrupado = pd.DataFrame(columns=['case:concept:name','Operations','Organization', 'Year', 'Question'])
for name, group in df_analisis:
    org = group.iloc[0]['Organization']
    year = group.iloc[0]['Year']
    question = group.iloc[0]['Question']
    lista = group['concept:name'].to_list()
        
    new_row={'case:concept:name':name, 'Operations':lista, 'Organization':org, 'Year':year, 'Question':question} 
    df_agrupado = df_agrupado.append(new_row, ignore_index=True)

In [10]:
df_agrupado

,case:concept:name,Operations,Organization,Year,Question
0,A1 P1 BPI 2017,"[Calculate cycle time, Calculate cycle time, F...",University of Liechtenstein\nVienna University...,2017,P1
1,A1 P1 BPI 2020,"[Filter traces, Calculate cycle time, Calculat...","Technische Universität, Berlin",2020,P1
2,A1 P2 BPI 2019,"[Group events by time, Calculate cycle time, R...",University of Melbourne,2019,P2
3,A1 P2 BPI 2020,"[Calculate cycle time, Compare cycle time]","Technische Universität, Berlin",2020,P2
4,A1 P5 BPI 2015,"[Calculate cycle time, Identify organizational...",Meijer & Van der Ham Management Consultants,2015,P5
...,...,...,...,...,...
121,A8 P1 BPI 2017,"[Filter traces, Calculate waiting time, Repres...",Universidade Federal do Estado do Rio de Janeiro,2017,P1
122,A8 P5 BPI 2015,"[Calculate cycle time, Compare cycle time, Gro...",GRADIENT ECM,2015,P5
123,A9 P1 BPI 2017,"[Calculate processing time, Calculate waiting ...",POSTECH,2017,P1
124,A9 P2 BPI 2019,"[Filter traces, Calculate cycle time, Identify...",KPMG Netherlands,2019,P2


In [11]:
def jaccardIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    tamIntersection=len(list(intersection))
    union = set1.union(set2)
    tamUnion=len(list(union))
#     print("similitud: "+str(float(tamIntersection)/tamUnion))
    return float(tamIntersection)/tamUnion

In [12]:
def overlapIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
#     print(float(len(list(intersection))))
#     print(min([len1,len2]))
    overlapIndex=float(len(list(intersection)))/min([len1,len2])
    return overlapIndex

In [13]:
def sorensenIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
    sorensenIndex=float(2*len(list(intersection)))/(len1+len2)
    return sorensenIndex

In [18]:
df_2020 = df_agrupado[df_agrupado['Year']==2020]
df2 = df_2020.groupby(by=['Year', 'Question'])
columns=['analisis','organizaciones','overlap','jaccard','sorensen']
df_metricas = pd.DataFrame(columns=columns)

for name, group in df2:
    for i in range(len(group)):
        if(group.iloc[i]['Organization'] != 'Sberbank'):
            org1 = 'No_Sberbank'
        else:
            org1 = 'Sberbank'
        
        for j in range(i+1,len(group)):
            grupo = group.iloc[i]['Operations']
            analisis = group.iloc[i]['case:concept:name'] + "-" + group.iloc[j]['case:concept:name']
                
            if(group.iloc[j]['Organization'] != 'Sberbank'):
                org2 = 'No_Sberbank'
            else:
                org2 = 'Sberbank'
            org = org1 + "-" + org2
            sorensen = sorensenIndex(grupo,group.iloc[j]['Operations'])
            overlap = overlapIndex(grupo,group.iloc[j]['Operations'])
            jaccard = jaccardIndex(grupo,group.iloc[j]['Operations'])
            new_row={'analisis':analisis,'organizaciones':org,'overlap':overlap,'jaccard':jaccard,'sorensen':sorensen} 
            df_metricas = df_metricas.append(new_row, ignore_index=True)
        
df_metricas = df_metricas.replace({'Sberbank-No_Sberbank':'No_Sberbank-Sberbank'})
df_metricas 

,analisis,organizaciones,overlap,jaccard,sorensen
0,A1 P1 BPI 2020-A10 P1 BPI 2020,No_Sberbank-No_Sberbank,0.333333,0.166667,0.285714
1,A1 P1 BPI 2020-A11 P1 BPI 2020,No_Sberbank-Sberbank,1.000000,0.500000,0.666667
2,A1 P1 BPI 2020-A12 P1 BPI 2020,No_Sberbank-Sberbank,0.333333,0.200000,0.333333
3,A1 P1 BPI 2020-A13 P1 BPI 2020,No_Sberbank-Sberbank,1.000000,0.428571,0.600000
4,A1 P1 BPI 2020-A16 P1 BPI 2020,No_Sberbank-No_Sberbank,0.666667,0.500000,0.666667
...,...,...,...,...,...
669,A30 P6 BPI 2020-A37 P6 BPI 2020,No_Sberbank-Sberbank,0.333333,0.142857,0.250000
670,A30 P6 BPI 2020-A7 P6 BPI 2020,No_Sberbank-No_Sberbank,0.666667,0.500000,0.666667
671,A35 P6 BPI 2020-A37 P6 BPI 2020,No_Sberbank-Sberbank,0.000000,0.000000,0.000000
672,A35 P6 BPI 2020-A7 P6 BPI 2020,No_Sberbank-No_Sberbank,0.500000,0.250000,0.400000


In [19]:
df_metricas.groupby(by=['organizaciones']).mean()

,overlap,jaccard,sorensen
organizaciones,,,
No_Sberbank-No_Sberbank,0.526381,0.280684,0.390306
No_Sberbank-Sberbank,0.491490,0.252058,0.349181
Sberbank-Sberbank,0.421026,0.205340,0.284500


In [20]:
from scipy import stats

group1 =  df_metricas[df_metricas['organizaciones'] == 'Sberbank-Sberbank']
group2 =  df_metricas[df_metricas['organizaciones'] == 'No_Sberbank-Sberbank']
group3 =  df_metricas[df_metricas['organizaciones'] == 'No_Sberbank-No_Sberbank']
# stats.kruskal(group1, group2, group3)

In [21]:
df_metricas.groupby(by=['organizaciones'])['overlap'].var(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.114360
No_Sberbank-Sberbank       0.139858
Sberbank-Sberbank          0.168820
Name: overlap, dtype: float64

In [22]:
df_metricas.groupby(by=['organizaciones'])['overlap'].std(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.338171
No_Sberbank-Sberbank       0.373976
Sberbank-Sberbank          0.410877
Name: overlap, dtype: float64

In [24]:
df_metricas.groupby(by=['organizaciones'])['jaccard'].var(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.057413
No_Sberbank-Sberbank       0.059655
Sberbank-Sberbank          0.057653
Name: jaccard, dtype: float64

In [25]:
df_metricas.groupby(by=['organizaciones'])['jaccard'].std(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.239610
No_Sberbank-Sberbank       0.244244
Sberbank-Sberbank          0.240111
Name: jaccard, dtype: float64

In [26]:
df_metricas.groupby(by=['organizaciones'])['sorensen'].var(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.069288
No_Sberbank-Sberbank       0.078767
Sberbank-Sberbank          0.083090
Name: sorensen, dtype: float64

In [27]:
df_metricas.groupby(by=['organizaciones'])['sorensen'].std(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.263226
No_Sberbank-Sberbank       0.280654
Sberbank-Sberbank          0.288253
Name: sorensen, dtype: float64